In [1]:
import json
import datetime
import pytz
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from statistics import mean
import statsmodels.api as sm
import datetime,time
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

In [2]:
path = "./ECE219_tweet_data/"
files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", "tweets_#nfl.txt", "tweets_#patriots.txt", "tweets_#sb49.txt", "tweets_#superbowl.txt"]
topics = ["gohawks", "gopatriots", "nfl", "patriots", "sb49", "superbowl"]
test_files =["sample0_period1.txt","sample0_period2.txt","sample0_period3.txt", "sample1_period1.txt","sample1_period2.txt","sample1_period3.txt","sample2_period1.txt","sample2_period2.txt","sample2_period3.txt"]
for i, fl in enumerate(files):
    print("files[" + str(i) + "] => " + fl)

files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


In [3]:
def parse_dataset(file):
    pst_tz = pytz.timezone('America/Los_Angeles')
    data_raw = []
    for line in open(path + file, 'r', encoding="utf-8") :
        row_tmp = []
        a = json.loads(line)
        time = a['citation_date']
        retweet = a['metrics']['citations']['total']
        foll = a['author']['followers']        
        row_tmp.append(time)
        row_tmp.append(retweet)
        row_tmp.append(foll)        
        data_raw.append(row_tmp)
    
    pddata_raw = pd.DataFrame(data_raw,columns=['time','retweets','followers'])
    pddata_raw = pddata_raw.sort_values(by = 'time')
    pddata_raw = pddata_raw.reset_index(drop=True)
    pddata_raw['tweets'] = 1    
    hour_accu = []
    hour_day = []
    for index, row in pddata_raw.iterrows():  
        p = datetime.datetime.fromtimestamp(row["time"], pst_tz)  
        hour_accu.append(((p.month-1)*31+p.day-14)*24+p.hour)
        hour_day.append(p.hour)    
    pddata_raw["time"] = hour_accu
    pddata_raw["hour of day"] = hour_day
    
    df = pd.DataFrame([],columns=['hour index','tweets','retweets','followers sum','followers max','hour of day'])
    df['hour index'] = range(pddata_raw.iloc[len(pddata_raw.index)-1,0]+1)
    df['tweets'] = pddata_raw.groupby("time")['tweets'].sum()
    df['retweets'] = pddata_raw.groupby("time")['retweets'].sum()
    df['followers sum'] = pddata_raw.groupby("time")['followers'].sum()
    df['followers max'] = pddata_raw.groupby("time")["followers"].max()
    df['hour of day'] = pddata_raw.groupby("time")['hour of day'].mean()
    df = df.drop([0]).fillna(0).reset_index(drop=True)

    df_y = df.iloc[1:,1].reset_index(drop=True)
    df = df[:len(df_y)] 
    return df.iloc[:,1:],df_y

In [4]:
def parse_dataset_ten_features(file):
    pst_tz = pytz.timezone('America/Los_Angeles')
    data_raw = []
    for line in open(path + file, 'r', encoding="utf-8") :
        row_tmp = []
        a = json.loads(line)
        citation_date = a['citation_date']
        retweet = a['metrics']['citations']['total']
        foll = a['author']['followers']             
        ment = len(a['tweet']['entities']['user_mentions'])        
        medi = len(a['tweet']['extended_entities']['media']) if 'extended_entities' in a['tweet'] else 0
        hist_tw = a['tweet']['user']["statuses_count"]
        hist_yr = a['tweet']['user']['created_at'][-4:]
        acti = hist_tw/(2015-float(hist_yr)+1) 
        auth = a['author']['name']
        favo = a['tweet']['user']['favourites_count']
        titl = len(a['title'])
        row_tmp.append(citation_date)
        row_tmp.append(retweet)
        row_tmp.append(foll)    
        row_tmp.append(ment) 
        row_tmp.append(medi) 
        row_tmp.append(acti)  
        row_tmp.append(auth)
        row_tmp.append(favo)
        row_tmp.append(titl)        
        data_raw.append(row_tmp)
    pddata_raw = pd.DataFrame(data_raw,columns=['time','retweets','followers','mentioned','media',\
                                                'active','author','favourites_count','title'])
    pddata_raw = pddata_raw.sort_values(by = 'time')
    pddata_raw = pddata_raw.reset_index(drop=True)
    pddata_raw['tweets'] = 1                   
    hour_accu = []
    hour_day = []
    for index, row in pddata_raw.iterrows():  
        p = datetime.datetime.fromtimestamp(row["time"], pst_tz)  
        hour_accu.append(((p.month-1)*31+p.day-14)*24+p.hour)
        hour_day.append(p.hour)    
    pddata_raw["time"] = hour_accu
    pddata_raw["hour of day"] = hour_day
    
    df = pd.DataFrame([],columns=['hour index','tweets','retweets','followers sum','followers max',\
                                  'mentioned','media','active','author','favourites_count','title'])
    df['hour index'] = range(pddata_raw.iloc[len(pddata_raw.index)-1,0]+1)
    df['tweets'] = pddata_raw.groupby("time")['tweets'].sum()
    df['retweets'] = pddata_raw.groupby("time")['retweets'].sum()
    df['followers sum'] = pddata_raw.groupby("time")['followers'].sum()
    df['followers max'] = pddata_raw.groupby("time")["followers"].max()
    df['mentioned'] = pddata_raw.groupby("time")['mentioned'].sum()
    df['media'] = pddata_raw.groupby("time")['media'].sum()
    df['active'] = pddata_raw.groupby("time")['active'].mean()  
    df['author'] = pddata_raw.groupby("time")['author'].nunique() # count number of not-repeating authors    
    df['favourites_count'] = pddata_raw.groupby("time")['favourites_count'].sum()
    df['title'] = pddata_raw.groupby("time")['title'].mean()
            
    df = df.drop([0]).fillna(0).reset_index(drop=True)
    df_y = df.iloc[1:,1].reset_index(drop=True)
    df = df[:len(df_y)]
    return df.iloc[:,1:],df_y

In [5]:
def plot_recipe(df_y, pred_y):
    plt.figure()
    area = np.pi * (4)**2/4
    plt.scatter(df_y, pred_y, s = area)
    plt.plot([df_y.min(), df_y.max()], [df_y.min(), df_y.max()], 'k--', lw = 1)
    plt.xlabel('true values')
    plt.ylabel('fitted values')
    plt.show()

In [6]:
def ols_regression(df, df_y):
    X2 = sm.add_constant(df)
    y = df_y.as_matrix()
    lm = sm.OLS(y, X2).fit()
    print(lm.summary())
    print(list(df))

In [7]:
#aggregate all data 
df, df_y = parse_dataset_ten_features(files[0])
for i in range(1,6):
    df_temp,df_temp_y=parse_dataset_ten_features(files[i])
    df=df.append(df_temp,ignore_index=True)
    df_y=df_y.append(df_temp_y,ignore_index=True)
    print(df_temp.shape,df.shape)

(574, 10) (1151, 10)
(585, 10) (1736, 10)
(585, 10) (2321, 10)
(585, 10) (2906, 10)
(585, 10) (3491, 10)


In [8]:
param_grid={'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
'max_features': ['auto', 'sqrt'],
'min_samples_leaf': [1, 2, 4],
'min_samples_split': [2, 5, 10],
'n_estimators': [200, 400, 600, 800, 1000,
1200, 1400, 1600, 1800, 2000]}
kf = KFold(n_splits=5,random_state=42,shuffle=True)

In [9]:
#RandomForestRegressor
avg_RMSE_train=np.zeros((8,2,3,3,10))
avg_RMSE_test=np.zeros((8,2,3,3,10))
avg_oob=np.zeros((8,2,3,3,10))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df):
                        X_train= df.iloc[train_index]
                        y_train= df_y.iloc[train_index]
                        X_test= df.iloc[test_index]
                        y_test= df_y.iloc[test_index]
                        reg = RandomForestRegressor(oob_score=True,n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, bootstrap=True,max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        oob_error.append(1 - reg.oob_score_)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    avg_oob[n1,n2,n3,n4,n5]=mean(oob_error)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE_test[n1,n2,n3,n4,n5],"oob=",avg_oob[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 1814.1988961675663 RMSE_test= 4707.9354801064 oob= 0.3569720887783383

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 1789.9145249017465 RMSE_test= 4720.4453203852245 oob= 0.3559795880799912

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 1755.7456254951921 RMSE_test= 4703.208313109233 oob= 0.35540310378120876

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 1744.8342036626393 RMSE_test= 4713.44074060564 oob= 0.35245053100065477

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 1748.1237162864045 RMSE_test= 4721.389148273278 oob= 0.3545565106734201

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 1751.8696

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1600
RMSE_train= 2620.4626406194106 RMSE_test= 4955.698533977563 oob= 0.3789231686815574

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1800
RMSE_train= 2620.906238937804 RMSE_test= 4964.02271017503 oob= 0.379087044413709

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 2000
RMSE_train= 2618.5178657518222 RMSE_test= 4960.862165637417 oob= 0.37857255275319096

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 200
RMSE_train= 3456.263428524737 RMSE_test= 4935.3731293243145 oob= 0.4066546645535882

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 400
RMSE_train= 3452.521576678109 RMSE_test= 4944.450776308151 oob= 0.40116476195185863

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 600
RMSE_train= 3437.58

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 1695.0224228042512 RMSE_test= 4453.634835820852 oob= 0.3368151083912539

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 1692.8916278961851 RMSE_test= 4458.226493845716 oob= 0.3345853295658896

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1400
RMSE_train= 1691.132704866957 RMSE_test= 4454.648106476469 oob= 0.33432932532245835

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1600
RMSE_train= 1697.6731201128716 RMSE_test= 4460.589134595095 oob= 0.33577776314217733

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1800
RMSE_train= 1697.9408190326083 RMSE_test= 4448.778163974289 oob= 0.33651680594576777

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 2000
RMSE_train= 169

max_depth= 10 max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 400
RMSE_train= 3522.172926310324 RMSE_test= 4819.799217526375 oob= 0.393332789246675

max_depth= 10 max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 600
RMSE_train= 3495.637307403448 RMSE_test= 4815.597919942167 oob= 0.39034970937815894

max_depth= 10 max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 800
RMSE_train= 3486.746764920033 RMSE_test= 4820.101049742313 oob= 0.38907331605527895

max_depth= 10 max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1000
RMSE_train= 3490.461127905198 RMSE_test= 4810.316515969999 oob= 0.38925804615467774

max_depth= 10 max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1200
RMSE_train= 3494.009564101282 RMSE_test= 4807.480275263875 oob= 0.38933030928228185

max_depth= 10 max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1400
RMSE_train= 3486

max_depth= 20 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1800
RMSE_train= 1757.9791714655303 RMSE_test= 4694.33017652395 oob= 0.35684419012314805

max_depth= 20 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 2000
RMSE_train= 1755.1791098238562 RMSE_test= 4696.7560122409095 oob= 0.3570607396261793

max_depth= 20 max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 200
RMSE_train= 2313.7425902179157 RMSE_test= 4864.085731914017 oob= 0.36718258298085654

max_depth= 20 max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 400
RMSE_train= 2300.8626581022936 RMSE_test= 4861.516641001348 oob= 0.3686258013864912

max_depth= 20 max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 600
RMSE_train= 2270.6305465328696 RMSE_test= 4847.1849789302705 oob= 0.3678461372619757

max_depth= 20 max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 800
RMSE_train= 2264.3

...

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 800
RMSE_train= 2773.645195118634 RMSE_test= 4713.361694513977 oob= 0.36564982064205676

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1000
RMSE_train= 2778.5255365557396 RMSE_test= 4705.418966045225 oob= 0.36693574527905526

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1200
RMSE_train= 2779.3758497428503 RMSE_test= 4697.877614366424 oob= 0.36599138643629714

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1400
RMSE_train= 2779.101543521054 RMSE_test= 4687.790720867271 oob= 0.36651262655040573

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1600
RMSE_train= 2777.927598960939 RMSE_test= 4689.4210293619835 oob= 0.3653557363729707

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1800
RMSE_train

max_depth= 200 max_features= sqrt min_samples_leaf= 4 min_samples_split= 10 n_estimators= 2000
RMSE_train= 3990.0330572438647 RMSE_test= 4996.079528828825 oob= 0.41212521160224336

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 1817.807775770445 RMSE_test= 4700.474368295067 oob= 0.35925213019242536

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 1791.6605394461876 RMSE_test= 4693.529857240384 oob= 0.3579487361463677

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 1752.0361079626423 RMSE_test= 4694.276177080169 oob= 0.35549376934632976

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 1742.10145748445 RMSE_test= 4713.893156544656 oob= 0.35300146255862985

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_tra

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1200
RMSE_train= 2623.5140134820963 RMSE_test= 4972.810162381721 oob= 0.379969290644316

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1400
RMSE_train= 2618.926234631433 RMSE_test= 4963.849373450307 oob= 0.379479731010519

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1600
RMSE_train= 2618.399100405764 RMSE_test= 4962.943296748686 oob= 0.37837956335398804

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1800
RMSE_train= 2617.2238729202923 RMSE_test= 4972.163489036311 oob= 0.3779487971013238

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 5 n_estimators= 2000
RMSE_train= 2615.0491913124592 RMSE_test= 4968.947289193621 oob= 0.37759894683414297

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 200
RMSE_tr

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 1748.6826868814026 RMSE_test= 4538.707037890154 oob= 0.3456347560805007

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 1706.4589859814 RMSE_test= 4516.512720874763 oob= 0.3412186155279058

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 1695.695900213524 RMSE_test= 4488.664891395316 oob= 0.3383277502426329

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 1695.9028807225075 RMSE_test= 4469.667619485904 oob= 0.33917339521490353

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 1700.9427699791263 RMSE_test= 4466.740545194431 oob= 0.33937988857339857

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1400
RMSE_train

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1600
RMSE_train= 2777.927598960939 RMSE_test= 4689.4210293619835 oob= 0.3653557363729707

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1800
RMSE_train= 2779.182655956648 RMSE_test= 4693.890576086253 oob= 0.36578473646662435

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 2000
RMSE_train= 2779.3349718979025 RMSE_test= 4696.744416914763 oob= 0.36534511631779504

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 200
RMSE_train= 3501.3152753375243 RMSE_test= 4810.747152930331 oob= 0.3930568750981107

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 400
RMSE_train= 3511.8738752031645 RMSE_test= 4828.629842942055 oob= 0.39258892138407214

max_depth= None max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 600
RMS

In [10]:
print("min oob =",np.min(avg_oob))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_oob), avg_oob.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("min RMSE in testset=",np.min(avg_RMSE_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE_test), avg_RMSE_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")
reg = RandomForestRegressor(oob_score=True,n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], bootstrap=True,max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df,df_y)
pred = reg.predict(df)
print("MSE=",mean_squared_error(df_y, pred))

min oob = 0.3325525295376627
parameters:
max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
min RMSE in testset= 4448.778163974289
parameters:
max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1800

MSE= 2634245.6964638233


In [11]:
#GradientBoostingRegressor
avg_RMSE1_train=np.zeros((8,2,3,3,10))
avg_RMSE1_test=np.zeros((8,2,3,3,10))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df):
                        X_train= df.iloc[train_index]
                        y_train= df_y.iloc[train_index]
                        X_test= df.iloc[test_index]
                        y_test= df_y.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE1_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE1_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE1_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE1_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 5.141715445523478 RMSE_test= 5395.9081579420235

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.6528196706645074 RMSE_test= 5395.932246911541

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.3669581095824263 RMSE_test= 5395.93434480288

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.3591412450614905 RMSE_test= 5395.934894339267

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.3589631520633751 RMSE_test= 5395.935023045235

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.35895857474646353 RMSE_test= 5395.935035743589

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_esti

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.38201372518958365 RMSE_test= 5711.900918297473

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1200
RMSE_train= 0.3611069924793406 RMSE_test= 5711.902921608326

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.35915928577348455 RMSE_test= 5711.903482113635

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1600
RMSE_train= 0.3589777199113105 RMSE_test= 5711.903695415956

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1800
RMSE_train= 0.3589600310276557 RMSE_test= 5711.903749904196



KeyboardInterrupt: 

In [ ]:
print("min RMSE in testset=",np.min(avg_RMSE1_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE1_test), avg_RMSE1_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df,df_y)
pred = reg.predict(df)
print("MSE=",mean_squared_error(df_y, pred))

# Question 9

In [13]:
import graphviz 
from sklearn import tree
feature_list0=['tweets','retweets','followers sum','followers max','mentioned','media','active','author','favourites_count','title']

In [14]:
forest1 =  RandomForestRegressor(oob_score=True,n_estimators=200,max_depth=10,min_samples_leaf=1,min_samples_split=2, bootstrap=True,max_features='sqrt',random_state=42)

X = df.iloc[:,:].values
y = df_y.iloc[:].values

forest1.fit(X, y)
tree1 = forest1.estimators_[1]
dot_data = tree.export_graphviz(tree1, out_file=None, feature_names=feature_list0, 
                                class_names="charges",   filled=True, rounded=True,  special_characters=True)
graph = graphviz.Source(dot_data)
graph.render('random_forest.png', view=True)

'random_forest.png.pdf'

In [15]:
ols_regression(df, df_y)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     1943.
Date:                Sat, 23 Mar 2019   Prob (F-statistic):               0.00
Time:                        18:07:38   Log-Likelihood:                -32962.
No. Observations:                3491   AIC:                         6.595e+04
Df Residuals:                    3480   BIC:                         6.601e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              148.8645    193.099  